# Entity Extratction with spaCy

## Overview

We're going to look for all the people mentioned in a pile of documents.

### Entites

"Entities" in documents are, generally, names -- names of people, places, and things such as companies. Finding out which entities are mentioned in a trove of documents can be pretty helpful, especially when you don't previously _know_ someone or some place is included the document.

There are services online that do this kind of extraction, including [DocumentCloud](https://www.documentcloud.org/) ([see how here](https://www.documentcloud.org/faq#faq-analyzing-1)), [Amazon Comprehend](https://aws.amazon.com/comprehend/features/) and [Google Natural Language](https://cloud.google.com/natural-language/).

### Using spaCy

We're going to do our entity extraction right here in our notebook using a pre-trained natural language model called [spaCy](https://spacy.io/). Specifically, we're using the spaCy [large English language model](https://spacy.io/models/en#en_core_web_lg) trained on the [OntoNotes dataset](https://catalog.ldc.upenn.edu/LDC2013T19) -- a trove of "telephone conversations, newswire, newsgroups, broadcast news, broadcast conversation, weblogs" that includes nearly 1.5 million English words.  

The spaCy project has a lot of great language features. We'll be looking at the [named entities feature](https://spacy.io/usage/linguistic-features#named-entities). Note also that there are [models for several languages](https://spacy.io/models) being developed in spaCy.


## The Plan

- We'll download the spaCy software and the large English language model.
- We'll also download a (smallish) pile of emails released in a court case.
- We'll learn how to use spaCy functions to extract entities
- We'll use the spaCy functions to scan all the pages of the emails.

## Credits

This notebook was written by John Keefe [Quartz](https://qz.com) at Quartz and includes document-processing code written included in [a blog post](https://qz.ai/discovering-interesting-documents-in-the-mauritius-leaks/) and a [Jupyter notebook](https://github.com/Quartz/aistudio-doc2vec-for-investigative-journalism/blob/master/Doc2vec%20for%20Investigative%20Journalism.ipynb) by Jeremy B. Merrill at Quartz, who used it to help find documents inside a document dump known as the [Mauritius Leaks](https://qz.com/1670632/how-quartz-used-ai-to-help-reporters-search-the-mauritius-leaks/).  

-- John Keefe, [Quartz](https://qz.com), October 2019

## Setup

### For those using Google Colaboratory ...

Be aware that Google Colab instances are ephemeral -- they vanish *Poof* when you close them, or after a period of sitting idle (currently 90 minutes), or if you use one for more than 12 hours.

Note that there's a part of this notebook that doesn't seem to work with a GPU (It's the vector part below) so we'll stay on the CPU for now.

### Everybody do this ...

Everyone needs to run the next cell, which initializes the Python libraries we'll use in this notebook.

In [ ]:
## *EVERYBODY* SHOULD RUN THIS CELL
## Running this can take 3-5 minutes ... but that's normal
## When it's done, you should see a line in green: 
## "✔ Download and installation successful"

!pip install -U spacy --quiet
!python -m spacy download en_core_web_lg
!pip install PyPDF2 --quiet

import spacy
import en_core_web_lg
import PyPDF2
import json
from os.path import exists

## The Data

In this tutorial, we're going to look at some emails from the office of New York City mayor Bill de Blasio that were released under the Freedom of Information Law. 

The emails were part of the ["Agent of the City" hubbub](https://www.ny1.com/nyc/all-boroughs/news/2018/05/24/agents-of-the-city-emails-released), in which 4,000 city emails were released. You can download the [original file here](https://a860-openrecords.nyc.gov/response/120252?token=c784372fd140497081b4bfcff9f0e3a0) -- though we'll be using a file containing just [the first 100 pages](https://qz-aistudio-public.s3.amazonaws.com/workshops/2018.05.24_BerlinRosen_Responsive_Records_100pgs.pdf) for this exercise. 

In [ ]:
# Run this cell to download the data we'll use for this exercise
!wget -N https://qz-aistudio-public.s3.amazonaws.com/workshops/deblasio_emails_data.zip --quiet
!unzip -q deblasio_emails_data.zip
print('Done!')

Let's look at what we have.

In [ ]:
%ls data/

## Trying spaCy's entity extraction feature

In [ ]:
# First we load the model into the notebook
nlp = en_core_web_lg.load()

In [ ]:
# Now let's give it a try
doc = nlp(u"San Francisco considers banning sidewalk delivery robots")


In [ ]:
for entity in doc.ents:
    print(entity.text, entity.label_, spacy.explain(entity.label_))

There's [a whole list of entities spaCy can detect](https://spacy.io/api/annotation#named-entities)!

In [ ]:
my_story = """
John drove his Volkswagen Golf north on Interstate 35 to Duluth, Minnesota,
where he stopped at the Aerial Lift Bridge and looked out over
Lake Superior. 
"""

doc = nlp(my_story)

## Load the emails into a "jsonl" file

JSONL is a file format that stores data in a JSON file, with each record living on its own line in the file.

This next block reads the PDF file and turns it into a JSONL file, which is much easier to work with.

In [ ]:
# read the PDF file into a new file called 'nyc_docs.jsonl'
jsonl_file = "nyc_docs.jsonl"
if not exists(jsonl_file):
    pdf_file = open('data/2018.05.24_BerlinRosen_Responsive_Records_100pgs.pdf', 'rb')
    read_pdf = PyPDF2.PdfFileReader(pdf_file)
    with open(jsonl_file, 'w') as f:
        for page_num in range(read_pdf.getNumPages()):
            page = read_pdf.getPage(page_num)
            page_content = page.extractText().encode('utf-8').decode("utf-8") 
            f.write(json.dumps({"_source": {"content": page_content}, "_id": f"p{page_num+1}"}) + "\n")

In [ ]:
# let's take a look at the first few lines of the file
!head nyc_docs.jsonl

Each line in the JSON file now represents a single page in the original document. So now we'll step through each line (aka page) and grab all the entities in the text. Then we'll print out all the entities.

## Finding and listing the names

In [ ]:
with open(jsonl_file, 'r') as f:        # open the jsonl file
    for line in f:                      # loop through each line ...
        line = json.loads(line)            # read the line 
        text = line["_source"]["content"]  # grab the text of the email
        page_number = line["_id"]          # grab the page number we're on
        doc = nlp(text)                    # load the text into the nlp model
        for ent in doc.ents:               # loop through each entity in the text...
            if (ent.label_ == "PERSON"):      # if the entity is a person's name ...
                print(page_number, ent.text)  # print the page number and the name

Really we want a list of _names_ not pages, right?

In [ ]:
list_of_names = {}

with open(jsonl_file, 'r') as f:
    for line in f:
        line = json.loads(line)
        text = line["_source"]["content"]
        page_number = line["_id"]
        doc = nlp(text)
        
        # loop through the entities in the page
        for ent in doc.ents:
            
            # is the entity is a person ...
            if (ent.label_ == "PERSON"):
                
                # check if we already have this entity
                if ent.text in list_of_names:
                    
                    # add this page to the entity's list of pages
                    list_of_names[ent.text] += " " + page_number
                    
                else:
                    
                    # otheriwise start a list of pages
                    list_of_names[ent.text] = page_number

In [ ]:
list_of_names

In [ ]:
for name, pages in sorted(list_of_names.items()):
    print(name, "(" + pages + ")" )

Once you know a name is _there_ then you can search for it [in the original document](https://qz-aistudio-public.s3.amazonaws.com/workshops/2018.05.24_BerlinRosen_Responsive_Records_100pgs.pdf).

# Detecting Document Similarity with spaCy

Now we'll use our pre-trained spaCy model to detect the _similarity_ each of the has with each of the other pages. This will allow us to find pages that are similarly-worded .

## The Data

All of the data and the installation steps above still apply.

## Credits

I found [this Medium post super helpful](https://medium.com/better-programming/the-beginners-guide-to-similarity-matching-using-spacy-782fc2922f7c), and used some of the code described there in the `process_text` function.

## Code

### First, let's clean up our data

We're going to remove "stop" words, such as _a_ and _the_. You can see all the stop words included in spaCy here:

In [ ]:
print(nlp.Defaults.stop_words)

Then we'll remove all the punctuation and pronouns. Plus some garbage tokens that came along with the emails we have. Everything gets cleaned up with this reusable function:

In [ ]:
# This function accepts text and returns cleaned-up text

def process_text(text):
    doc = nlp(text.lower()) # set the spaCy "doc" to the lower-case version of the text
    
    result = []  # we'll store our good tokens here
    
    for token in doc:   # loop through the tokens in the text
        
        if token.text in nlp.Defaults.stop_words: # skip (continue) if its a stop word
            continue
        if token.is_punct:  # skip (continue) if the token is a stop word
            continue
        if token.lemma_ == '-PRON-': # skip (continue) if its a pronoun word
            continue
        
        # the next two are meant to skip lots of the stray characters
        # in this particular data dump
            
        if "$" in token.text: # skip if token is or has a $
            continue
        if "#" in token.text: # skip if token is or has a #
            continue
            
        result.append(token.lemma_) # if we got here, add token to the list
        
    return " ".join(result)  # join all the remaining tokens with a space between them

In [ ]:
# This is my original text from earlier
my_story

In [ ]:
# Here's what it looks like if I clean it
my_story_cleaned = process_text(my_story)
my_story_cleaned

In [ ]:
# Here's what it's vector looks like! (it's 300 values)
doc = nlp(my_story_cleaned)

### Check the vector

Almost every token in the text will have an associated _vector_ of 300 values -- which is, in a way, its position in 300-dimentional space(!) For a series of words, or an entire page, spaCy averages all the token-vectors into one 300-value vector. [More about vectors in the spaCy documentation](https://spacy.io/usage/vectors-similarity).

Here's my little story, as a vector:

In [ ]:
print(doc.vector)

Now we'll loop through all of the lines in our file of emails, and for each one we'll clean the text and check the vector.

In [ ]:
list_of_vectors = []
list_of_text = []

with open(jsonl_file, 'r') as f:        # open the jsonl file
    for line in f:                      # loop through each line ...
        line = json.loads(line)            # read the line 
        text = line["_source"]["content"]  # grab the text of the email
        cleaned_text = process_text(text)  # clean up the text
        doc = nlp(cleaned_text)            # load the text into the nlp model
        list_of_vectors.append(doc.vector) # add the vector to the list of vectors
        list_of_text.append(line)          # add the page's info to a list, too
        print(cleaned_text)                # optionally printing each block of text
        print("---")

In [ ]:
# we need some extra libraries we'll load here:

import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
list_of_text[0]  # let's look at the first (0th) item 

In [ ]:
list_of_text[96] # and here's page 97, the 96th item

In [ ]:
list_of_vectors[96].shape  # This vector is 300 rows by one column.

### Cosine similarity

We're going to use "cosine similarity" to compare this one document to every other document in the set. Here are the [details on how to use the cosine_similarity function](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html). There's [more about cosine similarity here](https://www.machinelearningplus.com/nlp/cosine-similarity/).

In [ ]:
single_item = list_of_vectors[96].reshape(1,300) # reshape the vector to 1 row of 300 columns

similarities = cosine_similarity(single_item, list_of_vectors, dense_output=False)
similarities

Now in one step we sort those similarities lowest to highest (highest being the most similar) and print out the indexes of the best matches ...

In [ ]:
# note: need to use the 0th row here, even tho we only have one row
related_docs_indices = similarities[0].argsort()[:-5:-1]
print (related_docs_indices)

In [ ]:
# and here we print out the actual similarity values, higher = more similar
print (similarities[0, related_docs_indices])

In [ ]:
list_of_text[99]